In [1]:
import os
import json
import math
import numpy as np

In [2]:
annotation_dir = '/root/.darwin/datasets/sparrow-computing/speed_eval/releases/v2/annotations'
annotations_list = os.listdir(annotation_dir)

In [3]:
front_tire_x_list = []
front_tire_y_list = []
back_tire_x_list = []
back_tire_y_list = []
for i in range(34, 90):
    f = open(os.path.join(annotation_dir, f'{i}.json'))
    tires = json.load(f)
    if len(tires['annotations'])== 2:
        back_x = tires['annotations'][0]['keypoint']['x']
        back_y = tires['annotations'][0]['keypoint']['y']
        front_x = tires['annotations'][1]['keypoint']['x']
        front_y = tires['annotations'][1]['keypoint']['y']
        #
        front_tire_x_list.append(front_x)
        front_tire_y_list.append(front_y)
        back_tire_x_list.append(back_x)
        back_tire_y_list.append(back_y)
    if len(tires['annotations'])== 1:
        back_x = tires['annotations'][0]['keypoint']['x']
        back_y = tires['annotations'][0]['keypoint']['y']
        front_x = -1
        front_y = -1
        #
        front_tire_x_list.append(front_x)
        front_tire_y_list.append(front_y)
        back_tire_x_list.append(back_x)
        back_tire_y_list.append(back_y)

In [4]:
def get_distance(x1, y1, x2, y2):
    return math.sqrt((x1-x2)**2 + (y1 -y2)**2)

In [5]:
distance_list = []
for i in range(len(front_tire_x_list)):
    if front_tire_x_list[i] == -1:
        break
    for j in range(i,len(front_tire_x_list)):
        future_back_x = back_tire_x_list[j]
        future_back_y = back_tire_y_list[j]
        current_keypoints_distance = get_distance(back_x, back_y, front_x, front_y)
        future_keypoints_distance = get_distance(back_x, back_y, future_back_x, future_back_y)
        if  current_keypoints_distance <= future_keypoints_distance :    
            distance_list.append((i,j))